In [1]:
from datasets import load_dataset

In [2]:
ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized")
ds['train_prefs'][0]

{'prompt': 'how can i develop a habit of drawing daily',
 'prompt_id': '086b3e24f29b8956a01059f79c56db35d118a06fb6b844b095737d042795cd43',
 'chosen': [{'content': 'how can i develop a habit of drawing daily',
   'role': 'user'},
  {'content': "Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving yo

# Prepare dataset

In [3]:
import sys
import os
sys.path.insert(1, os.path.abspath( os.path.dirname('/Users/a.p.malakhov/Desktop/turbo-alignment/turbo_alignment')) )
from turbo_alignment.dataset.chat.chat import ChatDatasetRecord

/Users/a.p.malakhov/Desktop/turbo-alignment/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_settings" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [4]:
def convert_to_chat_record(row: dict[str, str | int]) -> ChatDatasetRecord:
    for msg in row['messages']:
        if msg['role'] == 'assistant':
            msg['role'] = 'bot'

    return ChatDatasetRecord(
        id=row['prompt_id'],
        messages=row['messages'][:-1],
    ).dict()

In [5]:
ds['train_chat_records'] = ds['train_prefs'].map(convert_to_chat_record, remove_columns=ds['train_prefs'].column_names)
ds['val_chat_records'] = ds['test_prefs'].map(convert_to_chat_record, remove_columns=ds['test_prefs'].column_names)

Map:   0%|          | 0/61135 [00:00<?, ? examples/s]

In [6]:
from turbo_alignment.common.data.io import write_jsonl
from pathlib import Path

write_jsonl([item for item in ds['train_chat_records']], Path('train_chat.jsonl'))
write_jsonl([item for item in ds['val_chat_records']], Path('val_chat.jsonl'))

# SFT model training

In [ ]:
!poetry run python -m turbo_alignment train_sft --experiment_settings_path turbo-alignment/tutorials/sft/sft.json